In [ ]:
import os
import sys

import pandas as pd

from ydata_profiling import ProfileReport

from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB

from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.preprocessing import MinMaxScaler, OrdinalEncoder
# from pandas.testing import assert_frame_equal

sys.path.append(os.path.abspath(".."))
from src.preprocessing import TitanicPreprocessor

import mlflow

import joblib


## Overview
### Este notebook contém as seguintes seções:
- Seção 1: EDA
- Seção 2: Feature Engineering
- Seção 3: Modelagem
- Seção 4: Preparação de pipeline de pre-processing para uso na api

### Seção 1 - EDA

Primeiramente, analisa-se o dataset que apresenta Ground Truth para entendermos a característica geral dos dados.

In [ ]:
gt_dataset = pd.read_csv('./data/train.csv')
gt_dataset.columns = gt_dataset.columns.str.lower()
gt_dataset

In [ ]:
profile = ProfileReport(gt_dataset,title='Titanic Dataset', explorative=True)
profile.to_file('./profile.html')

Com o auxílio do relatório, podemos já fazer algumas observações:
- 'cabin' tem um número excessivo de missing values, então será descartado
- há significativas correlações de 'survived' com: 'fare', 'pclass' e 'sex'
- várias colunas têm grande desbalanceamento, o que poderia ser tratado com, por exemplo, resampling numa análise mais aprofundada
- 'ticket' não tem qualquer missing value e tem valores repetidos, podendo assim ser utilizado com um proxy para famílias e ter maior sucesso nessa \
função de proxy quando comparado a se extrair o sobrenome da coluna 'name', já que pode haver sobrenomes populares, comuns a pessoas desconhecidas

### Seção 2 - Feature Engineering

Além de 'cabin', eliminamos aqui também 'passengerid' que é só um identificador (poderíamos fazer uma análise mais aprofundada para entender se a coluna relaciona de alguma forma com posição no barco, etc..)

In [ ]:
gt_dataset.drop(columns=['cabin','passengerid'],inplace=True)

Dado o contexto do dado, sabemos que pode ser uma boa ideia tratar a idade por grupos, pois a evacuação priorizava crianças. Assim, faremos um encoding por grupo de idade, com uma resolução de 10 anos.

In [ ]:
age_bins = [age_bin for age_bin in range(0, int(gt_dataset['age'].max())+10, 10)]

os.makedirs("pickle_files", exist_ok=True)
joblib.dump(age_bins, "pickle_files/age_bins.pkl")

gt_dataset["age_group"] = pd.cut(gt_dataset["age"], bins=age_bins, right=False).cat.codes
gt_dataset.drop(columns='age',inplace=True)
gt_dataset

Em relação ao ticket, vamos extrair o ticket count para atribuir a cada passageiro a informação de 'tamanho da família', já criar uma feature 'is_alone' para identificar se apenas aquele passageiro tem um determinado ticket e finalmente aplicar encoding para 'ticket'.

In [ ]:
gt_dataset["ticket_count"] = gt_dataset.groupby("ticket")["ticket"].transform("count")
gt_dataset["is_alone"] = (gt_dataset["ticket_count"] == 1)

ticket_encoder = OrdinalEncoder()
gt_dataset["ticket"] = ticket_encoder.fit_transform(gt_dataset[["ticket"]]).astype(int)

joblib.dump(ticket_encoder, "pickle_files/ticket_encoder.pkl")

# gt_dataset["ticket"] = pd.factorize(gt_dataset["ticket"])[0]

gt_dataset

Finalizando o feature engineering para esta análise simplificada, vamos aplicar mais alguns scalings, encodings e remover colunas que não interessam.

In [ ]:
cols_to_scale = ["sibsp", "parch", "fare", "ticket_count"]

scaler = MinMaxScaler()
gt_dataset[cols_to_scale] = scaler.fit_transform(gt_dataset[cols_to_scale])

joblib.dump(scaler, "pickle_files/scaler.pkl")

gt_dataset = pd.get_dummies(gt_dataset, columns=["sex", "embarked"], drop_first=True)
gt_dataset.drop(columns='name',inplace=True)
gt_dataset

### Seção 3 - Modelagem
Como se trata de um dataset Kaggle de competição, o test dataset não possui Ground Truth e não tem muita serventia para nós. Dessa forma, para tentar mitigar overfitting, vamos prosseguir com uma análise K-Fold no gt_dataset, para que os modelos sejam avaliados para diversos splits.

In [ ]:
X = gt_dataset.drop(columns=["survived"])
y = gt_dataset["survived"]

In [ ]:
models = {
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
    "Decision Tree": DecisionTreeClassifier(random_state=42),
    "SVM": SVC(kernel="rbf", probability=True),
    "Gradient Boosting": GradientBoostingClassifier(random_state=42),
    "KNN": KNeighborsClassifier(n_neighbors=5),
    "Naive Bayes": GaussianNB(),
    "Bagging": BaggingClassifier(random_state=42),
}

In [ ]:
# K-Fold
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

Agora instanciaremos o MLFlow para logar os modelos com suas métricas e metadados, e com uma automação simples salvaremos o que tiver melhor pontuação.

In [ ]:
mlflow.set_experiment("titanic-models")

In [ ]:
best_score = 0
best_model = None
best_model_name = ""


for name, model in models.items():
    with mlflow.start_run(run_name=name):
        # Avaliação por f1-score e K-Fold
        scores = cross_val_score(model, X, y, cv=cv, scoring="f1")
        mean_f1 = scores.mean()
        std_f1 = scores.std()

        # Treinamento
        model.fit(X, y)

        # Logging no MLFlow
        mlflow.log_params(model.get_params())

        mlflow.log_metric("f1_mean", mean_f1)
        mlflow.log_metric("f1_std", std_f1)

        mlflow.sklearn.log_model(
            sk_model=model,
            name=name,
            registered_model_name=f"{name.replace(' ', '_')}_Titanic"
        )

        print(f"{name} → f1-score médio: {mean_f1:.4f} (± {std_f1:.4f})")
        print("")

        if mean_f1 > best_score:
            best_score = mean_f1
            best_model = model
            best_model_name = name        

if best_model:
    filename = f"pickle_files/selected_model.pkl"
    joblib.dump(best_model, filename)
    print(f"\nMelhor modelo salvo em: {filename} (f1-score = {best_score:.4f})")            

### Seção 4: Pipeline de pré-processamento

Aqui com o intuito de padronizar o pré-processamento, exportamos a pipeline de pré-processamento pra o pickle file `preprocessor.pkl` que replica todos os passos feitos neste notebook, a ser utilizada pela API.

In [ ]:
pipeline_preproc = TitanicPreprocessor(scaler,ticket_encoder,age_bins)

In [ ]:
gt_dataset_pipeline = pd.read_csv('./data/train.csv')
gt_dataset_pipeline.columns = gt_dataset_pipeline.columns.str.lower()
gt_dataset_pipeline = pipeline_preproc.transform(gt_dataset_pipeline)
gt_dataset_pipeline

Confirmando que o dataset pré-processado utilizando pipeline é igual ao dataset pré-processado sem usar pipeline.

In [ ]:
if gt_dataset_pipeline.equals(gt_dataset.drop(columns='survived')):
    print("## Pipeline de pré-processamento validada! ##")
    joblib.dump(pipeline_preproc, "pickle_files/preprocessor.pkl")